This notebook brings uses the scripts read_data_features.py and read_data_genre_track.py to arrive at a cleaned dataset with ready to use features and labels.

The following is 'read_data_genre_track.py'. 

In [2]:
import numpy as np
import pandas as pd

In [93]:
import sklearn.preprocessing as skpp

In [3]:
metadata_path = 'fma_metadata'

In [4]:
## Read genre information
genres = pd.read_csv(metadata_path+'\\genres.csv')

In [5]:
genres

,genre_id,#tracks,parent,title,top_level
0,1,8693,38,Avant-Garde,38
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5
...,...,...,...,...,...
158,1032,60,102,Turkish,2
159,1060,30,46,Tango,2
160,1156,26,130,Fado,2
161,1193,72,763,Christmas,38


In [6]:
genre_grps = genres.groupby('parent')


In [7]:
genre_id_root = genre_grps.get_group(0)['genre_id'] # Extract the parent groups
print(genre_id_root)
type(genre_id_root)

1         2
2         3
3         4
4         5
7         8
8         9
9        10
11       12
12       13
13       14
14       15
16       17
19       20
20       21
31       38
162    1235
Name: genre_id, dtype: int64


pandas.core.series.Series

In [8]:
## Read track information
fields = ['track']
tracks = pd.read_csv(metadata_path+'\\tracks_mod.csv',nrows=85000)
tracks.head()

,track_id,genres,genres_all
0,2,[21],[21]
1,3,[21],[21]
2,5,[21],[21]
3,10,[10],[10]
4,20,"[76, 103]","[17, 10, 76, 103]"


In [9]:
tracks = tracks.dropna()

In [10]:
tgen = list(tracks['genres'])
#print(tgen)

In [11]:
tgen = [t.replace('[','') for t in tgen]
tgen = [t.replace(']','') for t in tgen]
tgen = [t.replace(' ','') for t in tgen]
#print(tgen)

In [12]:
for k in range(len(tgen)):
    if(len(tgen[k])!=0):
        tgen[k] = [i for i in map(int, tgen[k].split(','))]
    else:
        tgen[k]=[]
#print(tgen)

In [13]:
tracks['genres']=tgen

In [14]:
tracks.head()

,track_id,genres,genres_all
0,2,[21],[21]
1,3,[21],[21]
2,5,[21],[21]
3,10,[10],[10]
4,20,"[76, 103]","[17, 10, 76, 103]"


In [15]:
print(len(tgen))
print(tracks.index)

84991
Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            84990, 84991, 84992, 84993, 84994, 84995, 84996, 84997, 84998,
            84999],
           dtype='int64', length=84991)


In [16]:
## Match the tracks to the genre data: Create dictionary with genre label as key
genre_dict = {}
for i in range(len(tgen)): # for every track (tgen is a list of lists of genres corresponding to every track)
    if i in tracks.index:  
        ti = tracks['track_id'][i] # ti stores the track id corresponding to the index
        if len(tgen[i]) == 1: # new condition that checks if t has just one genre and only then proceeds
            for k in tgen[i]: # k iterates through list of genres of current track
                if k is not None:
                    if (k in genre_dict.keys()): # if new genre is in the keys of genre_dict
                        genre_dict[k]+=[ti] # append current track id to list of all track ids
                    else:
                        genre_dict.update({k:[ti]}) # if k is a new key, then add new element to dictionary with ti as value

In [17]:
## Total number of samples for the different root genres
for q in genre_id_root:
    print(q,len(genre_dict[q]))

2 119
3 101
4 320
5 731
8 543
9 16
10 505
12 1717
13 3
14 15
15 2221
17 1205
20 17
21 2363
38 1233
1235 199


In [18]:
type(genre_dict)
len(genre_dict)

137

The following is 'read_data_features.py'.

In [19]:
## Column information
fields = ['mfcc.40','mfcc.42','mfcc.43','mfcc.44','mfcc.45','mfcc.48','mfcc.49','mfcc.50',\
    'mfcc.53','mfcc.56','mfcc.120','mfcc.121','mfcc.122','mfcc.124','spectral_contrast.14',\
        'spectral_contrast.15','spectral_contrast.16','spectral_contrast.43',\
            'spectral_contrast.44','feature']


In [20]:
## Read feature information 
features = pd.read_csv(metadata_path+'\\features.csv',usecols=fields)
features

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,293,295,296,297,298,301,302,303,306,309,373,374,375,377,428,429,430,457,458) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,feature,mfcc.40,mfcc.42,mfcc.43,mfcc.44,mfcc.45,mfcc.48,mfcc.49,mfcc.50,mfcc.53,mfcc.56,mfcc.120,mfcc.121,mfcc.122,mfcc.124,spectral_contrast.14,spectral_contrast.15,spectral_contrast.16,spectral_contrast.43,spectral_contrast.44
0,statistics,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,std,std,std,std,mean,mean,mean,std,std
1,number,01,03,04,05,06,09,10,11,14,17,01,02,03,05,01,02,03,02,03
2,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,-1.6377296448e+02,-4.1753826141e+01,2.9144329071e+01,-1.5050157547e+01,1.8879371643e+01,-4.2531509399e+00,1.3597911596e+00,-2.6829998493e+00,-3.6553659439e+00,3.9982039928e+00,9.7809043884e+01,3.8569583893e+01,2.2576272964e+01,1.9869739532e+01,1.8005174637e+01,1.5363138199e+01,1.7129013062e+01,4.3217558861e+00,3.9362342358e+00
4,3,-1.5900416565e+02,-3.3233562469e+01,4.7342002869e+01,-6.2473182678e+00,3.1405355453e+01,-1.5958366394e+00,5.1327295303e+00,-3.4226787090e+00,-3.5288145542e+00,1.0904747248e+00,1.1168637085e+02,4.1191982269e+01,1.9406339645e+01,1.9330266953e+01,1.5732023239e+01,1.5046036720e+01,1.7374631882e+01,4.5170097351e+00,4.6319375038e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106572,155316,-182.829,-29.699,37.0939,8.70466,18.7214,3.82411,5.94148,-2.80135,-5.95988,-1.47516,76.5965,27.9972,25.3795,16.4726,24.3066,12.5278,14.8206,3.46352,3.47494
106573,155317,-228.317,-15.9733,35.3698,12.6348,17.2699,7.1899,8.51431,0.269748,-3.10916,-0.4354,96.1629,30.113,32.9106,16.5214,23.6643,12.8614,15.3714,3.9755,3.81202
106574,155318,-193.423,-33.3256,29.6756,4.63439,14.7397,3.68089,9.8148,-1.33572,-2.83849,-1.36057,68.8362,30.3845,22.1801,20.4124,22.8505,12.5656,15.1487,3.74728,3.59314
106575,155319,-180.515,-32.2863,40.0483,1.78348,19.4363,2.28135,9.78132,-2.47939,-4.38616,-1.03914,74.0105,25.688,16.0581,13.3727,22.7475,12.2031,14.5837,3.48932,3.52558


In [21]:
feat_temp = features['feature']
for i in range(3,len(feat_temp)): # looping through all track id's
    feat_temp[i]=int(feat_temp[i]) # converting track id's into integers
features['feature'] = feat_temp

In [22]:
## Include code to filter the rows based on the training id
temp_id_set =[2,3,5,10,20,26]
feat1 = features.loc[features['feature'].isin(temp_id_set)]
feat1

,feature,mfcc.40,mfcc.42,mfcc.43,mfcc.44,mfcc.45,mfcc.48,mfcc.49,mfcc.50,mfcc.53,mfcc.56,mfcc.120,mfcc.121,mfcc.122,mfcc.124,spectral_contrast.14,spectral_contrast.15,spectral_contrast.16,spectral_contrast.43,spectral_contrast.44
3,2,-1.6377296448e+02,-4.1753826141e+01,2.9144329071e+01,-1.5050157547e+01,1.8879371643e+01,-4.2531509399e+00,1.3597911596e+00,-2.6829998493e+00,-3.6553659439e+00,3.9982039928e+00,9.7809043884e+01,3.8569583893e+01,2.2576272964e+01,1.9869739532e+01,1.8005174637e+01,1.5363138199e+01,1.7129013062e+01,4.3217558861e+00,3.9362342358e+00
4,3,-1.5900416565e+02,-3.3233562469e+01,4.7342002869e+01,-6.2473182678e+00,3.1405355453e+01,-1.5958366394e+00,5.1327295303e+00,-3.4226787090e+00,-3.5288145542e+00,1.0904747248e+00,1.1168637085e+02,4.1191982269e+01,1.9406339645e+01,1.9330266953e+01,1.5732023239e+01,1.5046036720e+01,1.7374631882e+01,4.5170097351e+00,4.6319375038e+00
5,5,-2.0544049072e+02,-1.6085823059e+01,4.1514759064e+01,-7.6429538727e+00,1.6942802429e+01,5.0315696001e-01,8.6735134125e+00,-8.2713766098e+00,2.3778877258e+00,7.4419503212e+00,9.5049903870e+01,3.9367599487e+01,1.8873550415e+01,2.3159673691e+01,1.7097452164e+01,1.5969444275e+01,1.8646987915e+01,4.3763713837e+00,4.2627854347e+00
6,10,-1.3586482239e+02,-5.3453247070e+01,1.7198896408e+01,6.8680348396e+00,1.3934344292e+01,-5.1303811073e+00,2.3384548724e-01,-5.4212064743e+00,1.8441945314e+00,-5.5957680941e-01,1.0274355316e+02,4.4409843445e+01,2.9066904068e+01,1.5492753983e+01,1.9177480698e+01,1.4281867027e+01,1.5510050774e+01,4.0089564323e+00,3.6900355816e+00
7,20,-1.3513589478e+02,1.2354539871e+01,1.9764219284e+01,1.8670799255e+01,1.9643861771e+01,-2.2851834297e+00,8.8352098465e+00,-8.0546313524e-01,3.8759169579e+00,-5.7247143984e-01,5.4847156525e+01,2.2191858292e+01,1.5861686707e+01,1.2820264816e+01,1.6042890549e+01,1.6161235809e+01,1.7701635361e+01,4.3163709641e+00,3.9181377888e+00
8,26,-2.3312371826e+02,1.7931293488e+01,4.8448619843e+01,1.3609122276e+01,1.8967182159e+01,-4.8185664415e-01,3.2712149620e+00,1.6888108253e+00,-1.1078318357e+00,1.2559027411e-02,8.7970138550e+01,3.6071262360e+01,2.7250453949e+01,1.8444572449e+01,1.6500648499e+01,1.8051128387e+01,1.9152212143e+01,5.1144442558e+00,5.0920886993e+00


In [23]:
# Enter how many tracks to select from each genre
num_tracks_per_genre = 15

In [24]:
print(genre_id_root)
print(type(genre_id_root))

1         2
2         3
3         4
4         5
7         8
8         9
9        10
11       12
12       13
13       14
14       15
16       17
19       20
20       21
31       38
162    1235
Name: genre_id, dtype: int64
<class 'pandas.core.series.Series'>


In [25]:
rootg_tracks = np.zeros((16,16), dtype=int)
for i in genre_id_root:
    print(i)

2
3
4
5
8
9
10
12
13
14
15
17
20
21
38
1235


In [26]:
# convert pandas series to numpy array
gen_root_id_np = genre_id_root.to_numpy()

In [27]:
print(gen_root_id_np)

[   2    3    4    5    8    9   10   12   13   14   15   17   20   21
   38 1235]


In [29]:
rootg_tracks[:,0] = gen_root_id_np
#print(rootg_tracks)

In [30]:
for i in range(16):
    # access track_ids corresponding to parent genres: parent genre id's are in the first column of the nparray
    print(genre_dict[rootg_tracks[i][0]][:15])
    print("\n")
    if(len(genre_dict[rootg_tracks[i][0]][:15])==15):
        rootg_tracks[i,1:] = genre_dict[rootg_tracks[i][0]][:15]
# print the track id's associated with genre_id 2 - done!
# instead of passing 0th column entry of 0th row, pass 0th column entry of ith row - done!
# print the first 10 entries only instead of printing all the track ids's associated with the genre_id - done!
# after printing, append the same lists to the nparray. Append to ith row - done!
# Above gave an error because one of the genres had only 3 tracks. Insert if condition - done!

# So now the rootg_tracks should have all the relevant tracks. Print it to verify

[853, 1082, 1680, 1681, 1682, 1683, 1684, 1685, 1686, 1687, 1688, 1689, 3586, 3587, 3588]


[1042, 1043, 1044, 1045, 1046, 1047, 1048, 3424, 3425, 3426, 3427, 3428, 3429, 3430, 3431]


[144, 145, 146, 147, 237, 238, 1020, 1021, 1022, 1023, 1058, 1059, 1060, 1061, 1062]


[4850, 4851, 10026, 10027, 10028, 10029, 10030, 10031, 10032, 10033, 10034, 10035, 10036, 10037, 10038]


[5009, 5011, 5012, 5013, 5014, 5015, 5016, 5017, 5018, 5019, 5020, 5021, 5022, 5023, 5024]


[13190, 13976, 13978, 13999, 14898, 14899, 14900, 14901, 14902, 14906, 74429, 77571, 114300, 114301, 114302]


[10, 213, 821, 822, 823, 847, 1051, 1642, 1643, 1644, 1645, 1648, 1649, 1652, 1657]


[247, 249, 252, 255, 393, 394, 395, 520, 521, 522, 523, 524, 525, 526, 527]


[50953, 120465, 120466]


[9793, 9811, 19682, 28551, 55110, 59474, 59480, 59481, 59482, 59483, 59484, 59485, 59486, 59487, 86783]


[384, 385, 386, 387, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406]


[139, 140, 141, 142, 188, 189, 190, 191, 192

In [ ]:
print(rootg_tracks)
print(rootg_tracks.shape)

In [31]:
# Because genre 13 has only 3 tracks, we must remove that row altogether
rootg_tracks = np.delete(rootg_tracks, 8, 0)
print(rootg_tracks)
print(rootg_tracks.shape)

[[     2    853   1082   1680   1681   1682   1683   1684   1685   1686
    1687   1688   1689   3586   3587   3588]
 [     3   1042   1043   1044   1045   1046   1047   1048   3424   3425
    3426   3427   3428   3429   3430   3431]
 [     4    144    145    146    147    237    238   1020   1021   1022
    1023   1058   1059   1060   1061   1062]
 [     5   4850   4851  10026  10027  10028  10029  10030  10031  10032
   10033  10034  10035  10036  10037  10038]
 [     8   5009   5011   5012   5013   5014   5015   5016   5017   5018
    5019   5020   5021   5022   5023   5024]
 [     9  13190  13976  13978  13999  14898  14899  14900  14901  14902
   14906  74429  77571 114300 114301 114302]
 [    10     10    213    821    822    823    847   1051   1642   1643
    1644   1645   1648   1649   1652   1657]
 [    12    247    249    252    255    393    394    395    520    521
     522    523    524    525    526    527]
 [    14   9793   9811  19682  28551  55110  59474  59480  59481

### Our final array has 15 rows corresponding to 15 root genres and 16 columns, one for genre id and the others for tracks from that genre id.

In [ ]:
# now corresponding to these track id's, there are feature vectors, we need to extract them
# There are a total of 15x15 = 225 tracks
# so our final array will be of dimension 225 x n_f, where n_f is the number of features
# the following code extracts those rows from a features-like dataframe corresponding to the track_ids given in the list
temp_id_set =['2','3','5','10','20','26']
feat1 = features.loc[features['feature'].isin(temp_id_set)]
feat1

In [42]:
rootg_tracks.shape

(15, 16)

In [44]:
# from our rootg_tracks np array containing all the track_id's of interest, we must create a list with all the track_id's
# we create an nparray of 225 - done!
# now we append all our track_id's in groups of 15 to the np array - done!
all_tracks_training = rootg_tracks[:,1:].flatten()

In [45]:
# print all_tracks_training to verify
print(all_tracks_training)
print(all_tracks_training.shape)

[   853   1082   1680   1681   1682   1683   1684   1685   1686   1687
   1688   1689   3586   3587   3588   1042   1043   1044   1045   1046
   1047   1048   3424   3425   3426   3427   3428   3429   3430   3431
    144    145    146    147    237    238   1020   1021   1022   1023
   1058   1059   1060   1061   1062   4850   4851  10026  10027  10028
  10029  10030  10031  10032  10033  10034  10035  10036  10037  10038
   5009   5011   5012   5013   5014   5015   5016   5017   5018   5019
   5020   5021   5022   5023   5024  13190  13976  13978  13999  14898
  14899  14900  14901  14902  14906  74429  77571 114300 114301 114302
     10    213    821    822    823    847   1051   1642   1643   1644
   1645   1648   1649   1652   1657    247    249    252    255    393
    394    395    520    521    522    523    524    525    526    527
   9793   9811  19682  28551  55110  59474  59480  59481  59482  59483
  59484  59485  59486  59487  86783    384    385    386    387    396
    39

In [46]:
# convert the np array into a list of strings
list_all_tracks_training = all_tracks_training.tolist()
print(list_all_tracks_training)

[853, 1082, 1680, 1681, 1682, 1683, 1684, 1685, 1686, 1687, 1688, 1689, 3586, 3587, 3588, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 3424, 3425, 3426, 3427, 3428, 3429, 3430, 3431, 144, 145, 146, 147, 237, 238, 1020, 1021, 1022, 1023, 1058, 1059, 1060, 1061, 1062, 4850, 4851, 10026, 10027, 10028, 10029, 10030, 10031, 10032, 10033, 10034, 10035, 10036, 10037, 10038, 5009, 5011, 5012, 5013, 5014, 5015, 5016, 5017, 5018, 5019, 5020, 5021, 5022, 5023, 5024, 13190, 13976, 13978, 13999, 14898, 14899, 14900, 14901, 14902, 14906, 74429, 77571, 114300, 114301, 114302, 10, 213, 821, 822, 823, 847, 1051, 1642, 1643, 1644, 1645, 1648, 1649, 1652, 1657, 247, 249, 252, 255, 393, 394, 395, 520, 521, 522, 523, 524, 525, 526, 527, 9793, 9811, 19682, 28551, 55110, 59474, 59480, 59481, 59482, 59483, 59484, 59485, 59486, 59487, 86783, 384, 385, 386, 387, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 139, 140, 141, 142, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 6523, 6584, 6585, 6586, 

In [47]:
list_all_t_train_string = []
for i in list_all_tracks_training:
    list_all_t_train_string.append(str(i[0]))
print(list_all_t_train_string)
len(list_all_t_train_string)

TypeError: 'int' object is not subscriptable

In [50]:
all_relevant_training = features.loc[features['feature'].isin(list_all_tracks_training)]
all_relevant_training

,feature,mfcc.40,mfcc.42,mfcc.43,mfcc.44,mfcc.45,mfcc.48,mfcc.49,mfcc.50,mfcc.53,mfcc.56,mfcc.120,mfcc.121,mfcc.122,mfcc.124,spectral_contrast.14,spectral_contrast.15,spectral_contrast.16,spectral_contrast.43,spectral_contrast.44
3,2,-1.6377296448e+02,-4.1753826141e+01,2.9144329071e+01,-1.5050157547e+01,1.8879371643e+01,-4.2531509399e+00,1.3597911596e+00,-2.6829998493e+00,-3.6553659439e+00,3.9982039928e+00,9.7809043884e+01,3.8569583893e+01,2.2576272964e+01,1.9869739532e+01,1.8005174637e+01,1.5363138199e+01,1.7129013062e+01,4.3217558861e+00,3.9362342358e+00
4,3,-1.5900416565e+02,-3.3233562469e+01,4.7342002869e+01,-6.2473182678e+00,3.1405355453e+01,-1.5958366394e+00,5.1327295303e+00,-3.4226787090e+00,-3.5288145542e+00,1.0904747248e+00,1.1168637085e+02,4.1191982269e+01,1.9406339645e+01,1.9330266953e+01,1.5732023239e+01,1.5046036720e+01,1.7374631882e+01,4.5170097351e+00,4.6319375038e+00
5,5,-2.0544049072e+02,-1.6085823059e+01,4.1514759064e+01,-7.6429538727e+00,1.6942802429e+01,5.0315696001e-01,8.6735134125e+00,-8.2713766098e+00,2.3778877258e+00,7.4419503212e+00,9.5049903870e+01,3.9367599487e+01,1.8873550415e+01,2.3159673691e+01,1.7097452164e+01,1.5969444275e+01,1.8646987915e+01,4.3763713837e+00,4.2627854347e+00
6,10,-1.3586482239e+02,-5.3453247070e+01,1.7198896408e+01,6.8680348396e+00,1.3934344292e+01,-5.1303811073e+00,2.3384548724e-01,-5.4212064743e+00,1.8441945314e+00,-5.5957680941e-01,1.0274355316e+02,4.4409843445e+01,2.9066904068e+01,1.5492753983e+01,1.9177480698e+01,1.4281867027e+01,1.5510050774e+01,4.0089564323e+00,3.6900355816e+00
12,134,-2.0766148376e+02,-3.3416591644e+01,3.2260929108e+01,8.0747709274e+00,1.5349553108e+01,5.2458801270e+00,7.0094237328e+00,3.4019577503e+00,3.1349236965e+00,4.9092990160e-01,1.0277974701e+02,4.7266517639e+01,3.1092168808e+01,2.2690843582e+01,1.7004140854e+01,1.5552998543e+01,1.6775951385e+01,4.7156233788e+00,4.4409751892e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52962,77571,-126.019,-28.5822,16.0818,-9.06077,22.9742,7.97873,5.31811,-0.605724,1.09291,3.64969,91.9492,41.075,20.7612,19.2956,16.4279,14.194,17.0473,4.16639,4.31476
59226,86783,-251.061,-1.61983,35.4001,-1.80035,10.2632,0.289988,1.25477,3.41956,5.6763,3.96215,72.7334,38.4771,34.7999,15.5821,18.9506,20.8292,24.8095,5.05269,5.91837
75904,114300,-316.822,18.1904,14.8074,8.1518,1.56018,-6.49915,-6.57455,-6.83858,-7.44416,-4.86561,64.1487,42.066,25.0192,13.4527,20.8326,17.7995,19.314,4.49923,4.24777
75905,114301,-345.374,36.1077,14.8122,0.0115088,0.54063,-5.50734,-4.86474,-5.24791,-1.68629,-6.4425,78.3142,46.9169,26.6389,14.7972,19.7447,16.6529,19.5834,4.64781,4.51906


In [51]:
x_225 = all_relevant_training.to_numpy()

In [53]:
print(x_225)
print(x_225.shape)

[[2 '-1.6377296448e+02' '-4.1753826141e+01' ... '1.7129013062e+01'
  '4.3217558861e+00' '3.9362342358e+00']
 [3 '-1.5900416565e+02' '-3.3233562469e+01' ... '1.7374631882e+01'
  '4.5170097351e+00' '4.6319375038e+00']
 [5 '-2.0544049072e+02' '-1.6085823059e+01' ... '1.8646987915e+01'
  '4.3763713837e+00' '4.2627854347e+00']
 ...
 [114300 -316.82247925 18.190402985 ... 19.31404686 4.4992280006
  4.247774601000001]
 [114301 -345.37359619 36.107715606999996 ... 19.583353043 4.6478075981
  4.519056797]
 [114302 -337.02966309 19.878982544 ... 18.199241638 4.909409523
  4.5096421242]]
(225, 20)


In [54]:
print("The shape of our feature array is", x_225.shape)
print("The shape of our label array is", x_225.shape)

The shape of our feature array is (225, 20)


### To obtain set of one hot codes for our labels

In [68]:
# Our matrix with all label info is rootg_tracks
print(rootg_tracks.shape)
labels_map = rootg_tracks[:,0]
print(labels_map)
labels_mapped = [i for i in range(1,16)]
print(labels_mapped)
label_indicator = np.zeros((15,2), dtype=int)
label_indicator[:,0] = labels_map
label_indicator[:,1] = labels_mapped
# label_indicator helps us keep track of which label corresponds to which genre_id

(15, 16)
[   2    3    4    5    8    9   10   12   14   15   17   20   21   38
 1235]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [108]:
labels = np.zeros((225,1))
for i in range(15):
    labels[15*i:15*i+15,0] = i*np.ones((1,15))
print(len(labels))

225


In [116]:
num_classes = 15
one_hot = np.zeros((num_classes, len(labels)))
print(one_hot.shape)
for index_i, i in enumerate(labels):
    one_hot[int(i),index_i] = 1


(15, 225)


In [118]:
print(one_hot)
print(one_hot.shape)

[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 1.]]
(15, 225)


#### One hot labels have been generated